In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

from urllib.request import urlopen
from bs4 import BeautifulSoup
import time
import csv
import re

def create_analysis_link(link):
    new = 'https://finance.naver.com' + link
    return 'https://finance.naver.com' + link[:6] + 'coinfo' + link[10:]


links = [] # 최종적인 링크들을 담을 테이블
# 시가총액 1~200위 페이지
for page in range(1,37):
    html = urlopen('https://finance.naver.com/sise/sise_market_sum.naver?&page='+str(page))
    bs = BeautifulSoup(html, 'html.parser')
    rankings = bs.findAll('a',{'href' : re.compile('^(/item/main.)')})

    # 각 종목별 페이지로 연결
    for ranking in rankings:
        if 'href' in ranking.attrs:
            midLink = str(create_analysis_link(ranking.attrs['href']))

            # 종목별 페이지 속의 '재무분석' 페이지로 연결
            html = urlopen("{}".format(midLink))
            bs = BeautifulSoup(html,'html.parser')
            link = bs.find('div', {'class' : 'section inner_sub'}).find('iframe')

            if 'src' in link.attrs:
                financialLink = link.attrs['src']
                links.append(financialLink)

# for i in range(3) : (2019 + i)의 주가를 예측해보자!
for year in range(0, 3): 
# 순운전자본 구하기
    # 연도 하나 시작
    data = []
    stocks = []
    for one in links:
        # headless 설정
        options = webdriver.ChromeOptions()
        options.add_argument('headless')

        # 브라우저 생성
        browser = webdriver.Chrome(chrome_options=options)
        time.sleep(3)
        browser.get('{}'.format(one))

        try: # ETF를 걸러줄 장치
            # 종목 이름 구하기
            elementXpath = '//*[@id="pArea"]/div[1]/div/table/tbody/tr[1]/td/dl/dt[1]/span'
            title = browser.find_element(by=By.XPATH, value = elementXpath).text

            # 포괄손익계산서 이동하기
            browser.find_element_by_link_text("재무분석").click()
            browser.find_element_by_link_text("포괄손익계산서").click()

            time.sleep(1)
            # 영업이익 비교하기_ 조건 불합시 continue
            try:
                elementXpath = '//table[@summary="IFRS연결 연간 재무 정보를 제공합니다."][2]/tbody/tr[58]/td[{}]'.format(3+year)
                O_profitAfter = browser.find_element(by=By.XPATH, value = elementXpath).text

                elementXpath = '//table[@summary="IFRS연결 연간 재무 정보를 제공합니다."][2]/tbody/tr[58]/td[{}]'.format(2+year)
                O_profitBefore = browser.find_element(by=By.XPATH, value = elementXpath).text

                if O_profitAfter <= O_profitBefore:
                    browser.quit()
                    continue

            except: # 영업이익이 안뜬다면.
                browser.quit()
                continue

            # 재무상태표 이동하기
            browser.find_element_by_link_text("재무상태표").click()
            time.sleep(1)

            # 순운전자본 구하기
            elementXpath = '//table[@summary="IFRS연결 연간 재무 정보를 제공합니다."][2]/tbody/tr[2]/td[{}]'.format(3+year)
            current_asset = float(browser.find_element(by=By.XPATH, value = elementXpath).text.replace(",",""))

            elementXpath = '//table[@summary="IFRS연결 연간 재무 정보를 제공합니다."][2]/tbody/tr[128]/td[{}]'.format(3+year)
            current_liabilities = float(browser.find_element(by=By.XPATH, value = elementXpath).text.replace(",",""))

            elementXpath = '//table[@summary="IFRS연결 연간 재무 정보를 제공합니다."][2]/tbody/tr[47]/td[{}]'.format(3+year)
            cashable_asset = float(browser.find_element(by=By.XPATH, value = elementXpath).text.replace(",",""))

            net_working_capital = current_asset - current_liabilities + cashable_asset
            if net_working_capital < 0:
                continue

            # 조건으로부터 살아남았다면, 리스트 테이블에 추가
            data.append(net_working_capital) 
            stocks.append(title)
            browser.quit()

        except: # ETF는 거른다.
            browser.quit()
            continue

# 시가총액 구하기

    # 브라우저 생성 
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    browser = webdriver.Chrome(chrome_options=options)

    # 반복문 시작
    table = []
    stocks2 = []
    i = 0
    for stock in stocks:
        try:
            time.sleep(3)
            browser.get('{}'.format('http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020101'))

            # 원하는 곳으로 이동
            elementXpath = '//*[@id="jsMdiMenu"]/div[4]/ul/li[1]/ul/li[2]/div/div[1]/ul/li[2]/ul/li[1]/ul/li[3]/a'
            WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, elementXpath)))
            time.sleep(1)

            browser.find_element(by = By.XPATH , value = elementXpath).click()

            # 검색어 입력
            elementXpath = "//*[@id='tboxisuCd_finder_stkisu0_1']"
            WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, elementXpath)))
            time.sleep(1)

            search_box = browser.find_element(by = By.XPATH , value = elementXpath)
            browser.find_element(by = By.XPATH , value = elementXpath).clear()
            search_box.send_keys("{}".format(stock))
            search_box.send_keys(Keys.RETURN)

            # 새로운 창에서의 클릭
            browser.switch_to.window(browser.window_handles[-1])

            elementXpath = "//*[@id='jsGrid__finder_stkisu0_1']/tbody/tr[1]/td[2]"
            WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, elementXpath)))
            time.sleep(1)
            browser.find_element(by = By.XPATH , value = elementXpath).click()

            # 다시 원래 창으로 돌아옴.
            browser.switch_to.window(browser.window_handles[0])

            # 날짜 선택 먼저 20181231 + year
            box = browser.find_element(by = By.XPATH , value = " //*[@id='endDd']")
            box.clear()
            box.send_keys("20{}1231".format(18+year))

            # 1일 버튼 클릭
            browser.find_element(by = By.XPATH , value = "//*[@id='MDCSTAT017_FORM']/div[1]/div/table/tbody/tr[2]/td/div/div/button[2]").click()

            # '조회' 버튼 클릭
            browser.find_element(by = By.XPATH , value = '//*[@id="jsMdiTab"]/li[1]/a/button').click()
            browser.find_element(by = By.XPATH , value = "//*[@id='jsSearchButton']").click()

            # 출력 
            elementXpath = "//*[@id='jsMdiContent']/div/div[1]/div[1]/div[1]/div[2]/div/div/table/tbody/tr/td[10]"
            WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.XPATH, elementXpath)))
            time.sleep(1)

            cap = float(int(browser.find_element(by = By.XPATH , value = elementXpath).text.replace(",",""))/10**8)
            rate = round(cap/data[i]*100,2)
            if rate > 100:
                i += 1
                continue
            table.append([stock, str(round(rate,2))+'%'])
            stocks2.append(stock)
        except: # 혹여나 문제가 생기면 그냥 넘어가 
            i += 1
            continue

# 수익률 구하기
    csvFile = open('Cospi20{} 예측 자료.csv'.format(19+year), 'wt+', newline = '')
    writer = csv.writer(csvFile)

    writer.writerow(('기업 이름', '시가총액/순운전자본', '20{}/12 주가'.format(18+year), '2020/12 주가'.format(19+year), '수익률'))

    try:
        # 브라우저 생성
        options = webdriver.ChromeOptions()
        options.add_argument('headless')
        browser = webdriver.Chrome(chrome_options=options)

        i = 0
        for stock in arr:
            time.sleep(3)
            browser.get('{}'.format('http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020101'))

            # 원하는 곳으로 이동
            elementXpath = '//*[@id="jsMdiMenu"]/div[4]/ul/li[1]/ul/li[2]/div/div[1]/ul/li[2]/ul/li[1]/ul/li[4]/a'
            WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, elementXpath)))
            time.sleep(1)

            browser.find_element(by = By.XPATH , value = elementXpath).click()

            # 옵션 선택
            elementXpath = '//*[@id="MDCSTAT018_FORM"]/div[1]/div/table/tbody/tr[1]/td/label[2]'
            WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, elementXpath)))
            time.sleep(1)

            browser.find_element(by = By.XPATH , value = elementXpath).click()

            # 검색어 입력
            elementXpath = "//*[@id='tboxisuCd_finder_stkisu0_1']"    
            search_box = browser.find_element(by = By.XPATH , value = elementXpath)
            search_box.clear()
            search_box.send_keys("{}".format(stock))
            search_box.send_keys(Keys.RETURN)

            # 새로운 창에서의 클릭
            browser.switch_to.window(browser.window_handles[-1])

            elementXpath = "//*[@id='jsGrid__finder_stkisu0_1']/tbody/tr[1]/td[2]"
            WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, elementXpath)))
            time.sleep(1)

            browser.find_element(by = By.XPATH , value = elementXpath).click()

            # 다시 원래 창으로 돌아옴.
            browser.switch_to.window(browser.window_handles[0])
            
            # 드롭다운 날짜 선택
            select_fr = Select(browser.find_element(by = By.XPATH, value = '//*[@id="strtYear"]'))
            select_fr.select_by_index(5-year)
            select_fr = Select(browser.find_element(by = By.XPATH, value = '//*[@id="strtMonth"]'))
            select_fr.select_by_index(11)

            select_fr = Select(browser.find_element(by = By.XPATH, value = '//*[@id="endYear"]'))
            select_fr.select_by_index(4-year)
            select_fr = Select(browser.find_element(by = By.XPATH, value = '//*[@id="endMonth"]'))
            select_fr.select_by_index(11)

            # '조회' 버튼 클릭
            browser.find_element(by = By.XPATH , value = '//*[@id="jsMdiTab"]/li[1]/a/button').click()
            browser.find_element(by = By.XPATH , value = "//*[@id='jsSearchButton']").click()

            # 수익률
            try:
                elementXpath = '//*[@id="jsMdiContent"]/div/div[1]/div[1]/div[1]/div[2]/div/div/table/tbody/tr[1]/td[4]'
                WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.XPATH, elementXpath)))
                time.sleep(1)

                stock_before = float(browser.find_element(by = By.XPATH , value = elementXpath).text.replace(',',''))
            except:
                i += 1
                continue

            try:
                elementXpath = '//*[@id="jsMdiContent"]/div/div[1]/div[1]/div[1]/div[2]/div/div/table/tbody/tr[13]/td[4]'
                WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.XPATH, elementXpath)))
                time.sleep(1)

                stock_after = float(browser.find_element(by = By.XPATH , value = elementXpath).text.replace(',',''))
            except:
                i += 1
                continue

            earning_rate = str(round((stock_after-stock_before)/stock_before , 4)*100)+"%"
            table[i].append(int(stock_before))
            table[i].append(int(stock_after))
            table[i].append(earning_rate)
            writer.writerow(table[i])
            i += 1

    finally:
        csvFile.close()

In [ ]:
# 되면 대박!